In [19]:
from dotenv import load_dotenv
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage 
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from typing import Sequence
from typing_extensions import Annotated, TypedDict
from langgraph.graph.message import add_messages
from typing import Dict, Any

In [20]:
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    answer: str
    metadata: Dict[str,Any]

In [21]:
load_dotenv()
ZHIPU_API_KEY = os.getenv("ZHIPU_API_KEY")
llm = ChatZhipuAI(
    api_key=ZHIPU_API_KEY,
    temperature=0.5,
    model="glm-4-flash",
)

In [22]:
system_prompt = "你是一个为了软件架构而生的AI助手，辅助进行软件架构设计，你的名字是HIT软件架构小助手"
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [33]:
chain = prompt | llm


def call_model(state: State):
    print("state", state)
    response = chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response.content),
        ],
        "answer": response.content,
        "metadata": response.response_metadata,
    }


# Define a new graph
workflow = StateGraph(state_schema=State)

# Define the (single) node in the graph
workflow.add_edge(START, "llm")
# workflow.add_node("llm", call_model)


In [34]:
def add_history(state: State):
    state["chat_history"].append(HumanMessage("hello"))
    
# workflow.add_node("history_update", add_history)
workflow.add_node("llm", call_model)
# workflow.add_edge("history_update", "llm")
# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [35]:
config = {"configurable": {"thread_id": "aa22"}}
query = "你好，我叫Bob"

history = [HumanMessage(content='你好，我叫Bob', additional_kwargs={}, response_metadata={}, id='abaa0fcd-b785-4d8d-9586-70b690965781'), AIMessage(content='你好，Bob！', additional_kwargs={}, response_metadata={}, id='bd4ce4e1-63df-4273-a653-9304ad269107')]
result = app.invoke({'input':'1','chat_history':history},config=config)
print(result)

state {'input': '1', 'chat_history': [HumanMessage(content='你好，我叫Bob', additional_kwargs={}, response_metadata={}, id='abaa0fcd-b785-4d8d-9586-70b690965781'), AIMessage(content='你好，Bob！', additional_kwargs={}, response_metadata={}, id='bd4ce4e1-63df-4273-a653-9304ad269107')]}
{'input': '1', 'chat_history': [HumanMessage(content='你好，我叫Bob', additional_kwargs={}, response_metadata={}, id='abaa0fcd-b785-4d8d-9586-70b690965781'), AIMessage(content='你好，Bob！', additional_kwargs={}, response_metadata={}, id='bd4ce4e1-63df-4273-a653-9304ad269107'), HumanMessage(content='1', additional_kwargs={}, response_metadata={}, id='21c8ee34-2356-4eeb-9eeb-f0994f663fbb'), AIMessage(content='看起来你只发了一个数字“1”。如果你有任何关于软件架构的问题或者需要帮助，请随时告诉我！', additional_kwargs={}, response_metadata={}, id='0a0d308a-7ed9-47b5-83ef-cbf35d631e70')], 'answer': '看起来你只发了一个数字“1”。如果你有任何关于软件架构的问题或者需要帮助，请随时告诉我！', 'metadata': {'token_usage': {'completion_tokens': 25, 'prompt_tokens': 45, 'total_tokens': 70}, 'model_name': 'glm-4-flash', '

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}
history = app.get_state(config=config)
print(history)